This notebook takes the output from the lens modeling script as it is saved in OSG and compiles it. The output is a single png image per candidate. The output folder is called "summarized_models", and the current input directory is called "all_models/good_models".

In [1]:
import os
import re
import matplotlib.pyplot as plt
from matplotlib import gridspec
from PIL import Image
from tqdm import tqdm

In [8]:
"""
structure of output from OSG: 
- base path
    - ID
        - corner plot: ID_corner.png
        - image, residuals, & magnification: ID_model.png
        - MCMC chain trace plot: ID_TracePlot.png
        - parameter data: ID.txt
"""
# set current directory name
curr_dir = os.getcwd()

# make directory to store outputs in
output_dir = os.path.join(curr_dir, 'summarized_models')  # set output directory name
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# set base path for data
base_path = os.path.join(curr_dir, 'all_models', 'good_models')  # set input directory

# get list of all directories (IDs)
directories = list(os.listdir(base_path))
for d in directories:
    if d.startswith("."):
        directories.remove(d)



In [9]:
for ID in tqdm(directories):
    # reading image data
    corner_plot = Image.open(os.path.join(base_path, ID, ID+'_corner.png'))
    img = Image.open(os.path.join(base_path, ID, ID+'_model.png'))
    # trace = Image.open(os.path.join(base_path, ID, ID+'_TracePlot.png'))
    
    # reading text data
    f = open(os.path.join(base_path, ID, ID+'.txt'), 'r')
    lines = f.readlines()
    f.close()
    
    # get the reduced X^2 value and Einstein Rad.
    for line in lines:
        if line.startswith('Reduced X^2 of all data:'):
            chi_squared = round(float(re.findall(r'\d+\.\d+', line)[0]), 2)
            
        elif line.startswith('theta_E_lens0:'):
            theta_E = round(float(re.findall(r'\d+\.\d+', line)[0]), 2)
    
    plot_text = f'Einstein Radius: {theta_E}\nReduced X^2: {chi_squared}' 
     
    # create a figure
    fig = plt.figure()
    fig.set_figheight(10)
    fig.set_figwidth(20)

    # create grid for different subplots
    spec = gridspec.GridSpec(ncols=2, nrows=1,
                             width_ratios=[1, 1], wspace=0.0)
    
    ax0 = fig.add_subplot(spec[0])
    ax0.imshow(img)
    ax0.axis("off")
    
    
    ax1 = fig.add_subplot(spec[1])
    ax1.imshow(corner_plot)
    ax1.axis("off")
    ax1.text(1500, 200, plot_text, fontsize=18, bbox={
             'facecolor': 'grey', 'alpha': 0.5, 'pad': 10})
    
    # ax2 = fig.add_subplot(spec[2])
    # ax2.imshow(trace)
    # ax2.axis("off")
    
    # save the plots
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, ID+'.png'), format='png')
    # plt.show()
    plt.close()

100%|███████████████████████████████████████████████████| 931/931 [26:37<00:00,  1.72s/it]
